In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import FinalStreamingStdOutCallbackHandler

chat  = ChatOpenAI(
  temperature=0.1,
  streaming=True, 
  callbacks=[FinalStreamingStdOutCallbackHandler()]
)

template = PromptTemplate.from_template('What is the distance between {country_a} and {country_b} Also, what is your name?')
prompt = template.format(country_a='Mexico', country_b='Thailand')

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers). My name is Assistant.'

In [3]:

template = ChatPromptTemplate.from_messages([
  ("system", "You are a geography export. And you only reply in {language}."), 
  ("ai", "Ciao, mi chiamo {name}!"),
  ("human", "What is the distance between {country_a} and {country_b} Also, what is your name?")
])

prompt = template.format_messages(
  language='Greek',
  name="Socrates",
  country_a='Mexico',
  country_b='Thailand'
)
chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

from langchain.schema import BaseOutputParser 
class CommaOutputParser(BaseOutputParser):
  def parse(self, test):
    items = test.strip().split(',')
    return list(map(str.strip, items))
    

# p = CommaOutputParser()
# p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [6]:
template = ChatPromptTemplate.from_messages([
  ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase . Do NOT reply with anything else"), 
  ("human", "{question}"),
])

prompt = template.format_messages(
  max_items=10,
  question="What are the planets?"
)
result = chat.predict_messages(prompt)
p = CommaOutputParser()
p.parse(result.content)

['mercury', 'venus', 'earth', 'mars', 'jupiter', 'saturn', 'uranus', 'neptune']

In [8]:
chain = template | chat | CommaOutputParser()  # Chain! 
chain.invoke({
  "max_items": 10,
  "question": "What are the pokemons?"
})

['bulbasaur',
 'ivysaur',
 'venusaur',
 'charmander',
 'charmeleon',
 'charizard',
 'squirtle',
 'wartortle',
 'blastoise',
 'pikachu']

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat  = ChatOpenAI(
  temperature=0.1,
  streaming=True, 
  callbacks=[StreamingStdOutCallbackHandler()]
)
chef_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients."),
  ("human", "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat 

veg_chef_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."),
  ("human", "{recipe}"),
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain
final_chain.invoke({"cuisine": "indian",})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and delicious recipe for Chicken Tikka Masala. Here's how you can make it at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 teaspoons ground cumin
- 2 teaspoons ground coriander
- 1 teaspoon turmeric
- 1 teaspoon paprika
- 1 teaspoon garam masala
- 1 teaspoon salt
- 1 tablespoon vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon grated ginger
- 1 can (14 oz) crushed tomatoes
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)
- Cooked rice or naan bread (for serving)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, cumin, coriander, turmeric, paprika, garam masala, and salt. Add the chicken pieces and stir to coat. Cover and marinate in the refrigerator for at least 1 hour, or overnight for best results.

2. Preheat the oven to 400

AIMessageChunk(content="To make a vegetarian version of Chicken Tikka Masala, we can substitute the chicken with a plant-based alternative such as tofu or paneer. Here's how you can adapt the recipe:\n\nIngredients:\n- 1 lb firm tofu or paneer, cut into bite-sized pieces\n- 1 cup plain yogurt (you can use dairy-free yogurt for a vegan version)\n- 2 tablespoons lemon juice\n- 2 teaspoons ground cumin\n- 2 teaspoons ground coriander\n- 1 teaspoon turmeric\n- 1 teaspoon paprika\n- 1 teaspoon garam masala\n- 1 teaspoon salt\n- 1 tablespoon vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1 tablespoon grated ginger\n- 1 can (14 oz) crushed tomatoes\n- 1 cup coconut cream (or any other plant-based cream)\n- Fresh cilantro, chopped (for garnish)\n- Cooked rice or naan bread (for serving)\n\nInstructions:\n1. Follow the same marinating process as the original recipe, but instead of chicken, marinate the tofu or paneer in the yogurt and spice mixture. Let it marinate for a

In [40]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector


chat  = ChatOpenAI(
  temperature=0.1,
  streaming=True, 
  callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
  def __init__(self, examples):
    self.examples = examples

  def add_example(self, example):
    self.examples.append(example)

  def select_examples(self, input_variables):
    from random import choice
    return [choice(self.examples)]
    

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")
# example_selector = LengthBasedExampleSelector(
#   examples=examples,
#   example_prompt=example_prompt,
#   max_length=10
# )
example_selector = RandomExampleSelector(examples=examples)

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  example_selector=example_selector,
  suffix="Human: What do you know about {country}?", 
  input_variables=["country"]
)
prompt.format(country="Brazil")
# chain = prompt | chat 
# chain.invoke({"country": "Turkey"})


'Human: What do you know about France?\nAI: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat  = ChatOpenAI(
  temperature=0.1,
  streaming=True, 
  callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
  {
  "question": "France",
  "answer": """
  Here is what I know:
  Capital: Paris
  Language: French
  Food: Wine and Cheese
  Currency: Euro
  """,
  },
  {
  "question": "Italy",
  "answer": """
  I know this:
  Capital: Rome
  Language: Italian
  Food: Pizza and Pasta
  Currency: Euro
  """,
  },
  {
  "question": "Greece",
  "answer": """
  I know this:
  Capital: Athens
  Language: Greek
  Food: Souvlaki and Feta Cheese
  Currency: Euro
  """,
  },
]


example_prompt = ChatPromptTemplate.from_messages([
  ("human", "What do you know about {question}?"),
  ("ai", "{answer}")
  
])
example_prompt = FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt, 
  examples=examples
)
final_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a geography expert. you give short answers."), 
  example_prompt, 
  ("human", "what do you know about {country}?")
])
chain = final_prompt | chat 
chain.invoke({"country": "Thailand"})



  I know this:
  Capital: Bangkok
  Language: Thai
  Food: Pad Thai and Tom Yum Soup
  Currency: Thai Baht

AIMessageChunk(content='\n  I know this:\n  Capital: Bangkok\n  Language: Thai\n  Food: Pad Thai and Tom Yum Soup\n  Currency: Thai Baht')

In [47]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt
from langchain.prompts.pipeline import PipelinePromptTemplate

# prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")
chat  = ChatOpenAI(
  temperature=0.1,
  streaming=True, 
  callbacks=[StreamingStdOutCallbackHandler()]
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}
                                  
    {start}
"""
)

prompts = [
  ("intro", intro), 
  ("example", example), 
  ("start", start)
]

full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)

chain = full_prompt | chat

chain.invoke({
  "character":"pilot",
  "example_question":"What is your location?",
  "example_answer":"Arrrrg! That is a secret!! Arg arg!!",
  "question":"What is your favorite food?"
})

I love a good steak after a long flight, mate! Nothing beats a juicy steak with some mashed potatoes and gravy. Yarrr!

AIMessageChunk(content='I love a good steak after a long flight, mate! Nothing beats a juicy steak with some mashed potatoes and gravy. Yarrr!')

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache('cache.db'))
# set_debug(True)

chat = ChatOpenAI(
  temperature=0.1
)

chat.predict("How do you make italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into desired shapes, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes, or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'

In [2]:
chat.predict("How do you make italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into desired shapes, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes, or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
  temperature=0.1
)

with get_openai_callback() as usage: 
  a = chat.predict("What is the recipe for soju?")
  b = chat.predict("What is the recipe for bread?")
  print(a, b, "\n")
  print(usage)

Ingredients:
- 1 cup of rice
- 1 cup of water
- 1 tablespoon of nuruk (fermentation starter)
- 1 tablespoon of yeast

Instructions:
1. Rinse the rice under cold water until the water runs clear.
2. In a large pot, combine the rice and water and bring to a boil. Reduce heat to low and simmer for 20 minutes, or until the rice is cooked.
3. Remove the pot from heat and let it cool to room temperature.
4. In a separate bowl, mix the nuruk and yeast with a little bit of warm water to form a paste.
5. Add the paste to the cooled rice and mix well.
6. Transfer the mixture to a clean glass jar and cover with a clean cloth.
7. Let the mixture ferment in a dark, cool place for 1-2 weeks, stirring occasionally.
8. After fermentation is complete, strain the mixture through a cheesecloth to remove any solids.
9. Transfer the liquid to a clean bottle and store in the refrigerator.
10. Serve chilled and enjoy your homemade soju! Ingredients:
- 3 1/2 cups all-purpose flour
- 1 packet active dry yeast
